# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([66])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 0:
        x.append(example)
        y.append(-1)
    elif label == 1:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    unitary(params[0:6], 1, 2)
    unitary(params[6:12], 3, 4)
    unitary(params[12:18], 5, 6)

    # Second layer
    unitary(params[18:24], 0, 1)
    unitary(params[24:30], 2, 3)
    unitary(params[30:36], 4, 5)
    unitary(params[36:42], 6, 7)

    # Third layer
    unitary(params[42:48], 2, 5)

    # Fourth layer
    unitary(params[48:54], 1, 2)
    unitary(params[54:60], 5, 6)

    # Fifth layer
    unitary(params[60:66], 2, 5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549]

Example features: [-3.63840234 -2.87217609 -1.38022087 -0.4780986  -0.25409499  0.27703028
  0.07474095  0.079672  ]

Expectation value: 0.08266765070062759

 0: ──RY(-3.64)───RY(0.224)─────────────────────────────────────────────────────────

# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.552901023890785
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.7872582480091013
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.8179749715585893
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.8213879408418657
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.8498293515358362
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.8441410693970421
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.8475540386803185
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.8430034129692833
epoch 71
epoch 72
epoch 73
epo

epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.8794084186575655
test_accuracies: [0.8475540386803185, 0.8430034129692833, 0.8646188850967008, 0.8373151308304891, 0.8748577929465302, 0.870307167235495, 0.8589306029579067, 0.856655290102389, 0.8521046643913538, 0.8714448236632537, 0.8748577929465302, 0.8657565415244596, 0.8907849829351536, 0.8668941979522184, 0.8691695108077361, 0.8680318543799772, 0.8771331058020477, 0.870307167235495, 0.8714448236632537, 0.8748577929465302, 0.8782707622298066, 0.8725824800910125, 0.870307167235495, 0.8714448236632537, 0.870307167235495, 0.8725824800910125, 0.8714448236632537, 0.8805460750853242, 0.875995449374289, 0.8794084186575655]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.89419795221843
best accuracy so far!
test_accuracies: [0.8430034129692833, 0.8646188850967008, 0.8373151308304891, 0.8748577929465302, 0.870307167235495, 0.8589306029579067, 0.85665529010

epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.8794084186575655
test_accuracies: [0.8657565415244596, 0.8907849829351536, 0.8668941979522184, 0.8691695108077361, 0.8680318543799772, 0.8771331058020477, 0.870307167235495, 0.8714448236632537, 0.8748577929465302, 0.8782707622298066, 0.8725824800910125, 0.870307167235495, 0.8714448236632537, 0.870307167235495, 0.8725824800910125, 0.8714448236632537, 0.8805460750853242, 0.875995449374289, 0.8794084186575655, 0.89419795221843, 0.8691695108077361, 0.8850967007963595, 0.8680318543799772, 0.8714448236632537, 0.8737201365187713, 0.8919226393629124, 0.8919226393629124, 0.8953356086461889, 0.8714448236632537, 0.8794084186575655]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.8919226393629124
test_accuracies: [0.8907849829351536, 0.8668941979522184, 0.8691695108077361, 0.8680318543799772, 0.8771331058020477, 0.870307167235495, 0.8714448236632537, 0.8748577929465302, 0.87827076222

epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.8907849829351536
test_accuracies: [0.8714448236632537, 0.870307167235495, 0.8725824800910125, 0.8714448236632537, 0.8805460750853242, 0.875995449374289, 0.8794084186575655, 0.89419795221843, 0.8691695108077361, 0.8850967007963595, 0.8680318543799772, 0.8714448236632537, 0.8737201365187713, 0.8919226393629124, 0.8919226393629124, 0.8953356086461889, 0.8714448236632537, 0.8794084186575655, 0.8919226393629124, 0.89419795221843, 0.8805460750853242, 0.875995449374289, 0.8850967007963595, 0.881683731513083, 0.881683731513083, 0.875995449374289, 0.8725824800910125, 0.8850967007963595, 0.8907849829351536, 0.8907849829351536]
epoch 581
epoch 582
epoch 583
epoch 584
epoch 585
epoch 586
epoch 587
epoch 588
epoch 589
epoch 590
accuracy: 0.8805460750853242
test_accuracies: [0.870307167235495, 0.8725824800910125, 0.8714448236632537, 0.8805460750853242, 0.875995449374289, 0.8794084186575655

In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 1.27708731  0.0418406   1.17656788  0.89938174  1.45642609 -0.49868312
  0.37890149 -0.02206522  0.12904403  0.07320608  0.94279088  0.53608522
 -1.75211862  0.43277391  0.85323566  0.97988847  0.85899252 -0.23636722
 -0.41831644  1.35017644 -0.27600096  0.04688274  0.66133746  0.24953542
 -0.305001    0.32282209  1.11461573  1.81793655  0.21440776  0.23132314
  0.75891848  0.44127749 -0.20394899  0.07579767  0.72117264 -0.1991202
  0.45284395 -1.22641762  0.11191774 -0.27693122 -0.04148221  0.16187284
  0.19171052  0.13225081  0.85987269  1.44920939  0.67502418  1.28414991
  0.64997733  0.0280585   1.37091801  1.84556485  0.58058546  0.47868408
  1.01758294 -0.22883824  1.58664478 -0.66013907  1.09476426  0.76905057
  0.37273229  0.75073109  0.00424176  0.63528275  0.11683869  0.06182167]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.878213802435724


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.911926031112671
training time: 2276.3546538352966
test time: 49.9194598197937
total time: 2329.186039686203
